# Accounting Automated

Author: James A. Nguyen

Course Project, UC Irvine, Math 10, Summer 2023

## Introduction

As a student, I also work as a part-time intern, a job that requires little to no knowledge about coding outside of 'proficiency in excel.' However, my redundant workflow reminded me of the DRY principle, "Don't Repeat Yourself." My job requires me to analyze invoices and determine to which of our company's checking accounts it should be charged. By reviewing my workflow, I was able to use my experience to determing what I believe to be solid predictors of an invoice's account classification.

## Preparing the dataset

In [1]:
import numpy as np
import pandas as pd
import altair as alt
df_pre = pd.read_csv("Invoice Register Dataset.csv").sample(5000, random_state = 93361984)
df_pre.head()

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3258: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Payee Code,Payee Name,Description,Control,Property,Invoice Date,Payment Method,Invoice,Account,Invoice #,...,Due Date,Unpaid,Payment/,Payment/.1,Due To / Due From,Receivables,Original Source,Immediate Source,Misc Segments,Expense Department
9474,NaN,NaN,CAB HINGES,P-43622,oakv,8/18/2017,Check,Aug-17,5760-000 Maintenance Supplies,9156775791,...,9/17/2017,0,1405.0,8/31/2017,NaN,NaN,NaN,NaN,NaN,NaN
96874,NaN,NaN,"Hotpoint 17.5 Cu Ft Tp Mnt Rfrgrtr Wht, SKU=50...",P-417964,freewest,1/13/2023,Check,Feb-23,1557-000 Replacement Refrigerator,9210455168,...,2/12/2023,0,3682.0,2/22/2023,NaN,NaN,NaN,NaN,NaN,NaN
21291,NaN,NaN,Kidde photoelectric smoke alarm 10 yr btry,P-101737,fairmont,1/16/2019,Check,Jan-19,5725-000 R&M - Fire Life Safety,9169347186,...,2/15/2019,0,1396.0,4/17/2019,NaN,NaN,NaN,NaN,NaN,NaN
88156,NaN,NaN,2-stroke Rear Handle 18-inch Chain Saw,P-378749,ridgeapt,9/22/2022,Check,Oct-22,5886-000 Tools & Equipment,9207115074,...,10/22/2022,0,266.0,10/20/2022,NaN,NaN,NaN,NaN,NaN,NaN
36614,NaN,NaN,FILL VALVE AND FLAPPER,P-154905,grandpnt,12/17/2019,Check,Jan-20,5745-000 R&M - Plumbing,9178051085,...,1/16/2020,0,2272.0,1/15/2020,NaN,NaN,NaN,NaN,NaN,NaN


Taking a look at the columns of the dataset, we can eliminate any columns that may not be useful in making predictions. Further, we can drop any missing values from the sub-dataframe.

In [2]:
df = df_pre[["Description","Account"]].copy().dropna()

For the same of this project, we will only use data that contains the substring `'SKU,'` as this value will be useful later.
In order to make use of this data, though, we need to isolate it first, by separating each description into a list of strings.

In [3]:
df["Description"] = df["Description"].str.split()
df

,Description,Account
9474,"[CAB, HINGES]",5760-000 Maintenance Supplies
96874,"[Hotpoint, 17.5, Cu, Ft, Tp, Mnt, Rfrgrtr, Wht...",1557-000 Replacement Refrigerator
21291,"[Kidde, photoelectric, smoke, alarm, 10, yr, b...",5725-000 R&M - Fire Life Safety
88156,"[2-stroke, Rear, Handle, 18-inch, Chain, Saw]",5886-000 Tools & Equipment
36614,"[FILL, VALVE, AND, FLAPPER]",5745-000 R&M - Plumbing
...,...,...
13114,"[63x60, white, 3-1/2"", vert, blinds]",5623-000 Window Coverings
7626,"[Bottled, water]",5410-000 Office Supplies & Expense
6305,"[Kitchen, faucets]",5745-000 R&M - Plumbing
51899,"[18W, 11"", LED, FlshMnt, 90CRI, Nickel, 2/Pkg]",5750-000 R&M - Electrical


We can isolate the `'SKU'` value by locating each string containing `'SKU'` within each row's description, and moving it to a new column, `df["SKU"]`

In [4]:
sku = []
for i in range(len(df)):
    current = [df["Description"].iloc[i][j][4:] for j in range(len(df["Description"].iloc[i])) if "SKU" in df["Description"].iloc[i][j]]
    sku.append(current)
df["SKU"] = [a for a in sku]
df

,Description,Account,SKU
9474,"[CAB, HINGES]",5760-000 Maintenance Supplies,[]
96874,"[Hotpoint, 17.5, Cu, Ft, Tp, Mnt, Rfrgrtr, Wht...",1557-000 Replacement Refrigerator,[501113]
21291,"[Kidde, photoelectric, smoke, alarm, 10, yr, b...",5725-000 R&M - Fire Life Safety,[]
88156,"[2-stroke, Rear, Handle, 18-inch, Chain, Saw]",5886-000 Tools & Equipment,[]
36614,"[FILL, VALVE, AND, FLAPPER]",5745-000 R&M - Plumbing,[]
...,...,...,...
13114,"[63x60, white, 3-1/2"", vert, blinds]",5623-000 Window Coverings,[]
7626,"[Bottled, water]",5410-000 Office Supplies & Expense,[]
6305,"[Kitchen, faucets]",5745-000 R&M - Plumbing,[]
51899,"[18W, 11"", LED, FlshMnt, 90CRI, Nickel, 2/Pkg]",5750-000 R&M - Electrical,[]


Now, all we have to do to get only rows with a `'SKU'` number is use the `.drop()` modifier, indexed by a boolean array of the truth values associated with whether a row contains the empty string`'[]'`

In [5]:
df = df.drop([i for i in df.index if df["SKU"].loc[i]==[]])
df

,Description,Account,SKU
96874,"[Hotpoint, 17.5, Cu, Ft, Tp, Mnt, Rfrgrtr, Wht...",1557-000 Replacement Refrigerator,[501113]
93694,"[Sharkbite, Flexible, Connector, 18"", 3/4"",, S...",1573-000 Replacement Boiler/Water Heater,[402511]
89268,"[Universal, Mbox, Lock, Clip, On, Cylinder,, S...",5712-000 R&M - Hardware Locks & Keys,[925197]
100468,"[Edge, Banded, Door, and, Drawer, Fronts,, SKU...",5660-000 R&M - Windows & Doors,[597509]
79051,"[Sylvania, Flr, Bulb, 32w, T8, 5000k, Ll, 30/P...",5750-000 R&M - Electrical,[309113]
...,...,...,...
85672,"[30""white, Non, Vented, Range, Hood, W, Bracke...",5755-000 R&M - Appliances,[281200]
86456,"[Kwikset, Tylo, Passage, Knob, Satin, NICKEL,,...",5712-000 R&M - Hardware Locks & Keys,[817744]
110030,"[1000, Ml, PURELL, Nxt, Gel, Refill,, SKU=117890]",5605-000 Janitorial Service & Supplies,[117890]
75448,"[Two, Handle, Kitchen, Faucet, Cp, Dckplt, Chr...",5745-000 R&M - Plumbing,[158070]


An issue arises, though, that some `'SKU'` values contain strings such as ['MISC'], ['SHIPFLA'], or ["]. We can drop rows with a SKU string that does not have a lentgth of exactly 6.

In [6]:
df = df.drop([i for i in df.index if len(df["SKU"].loc[i][0])!=6])
df

,Description,Account,SKU
96874,"[Hotpoint, 17.5, Cu, Ft, Tp, Mnt, Rfrgrtr, Wht...",1557-000 Replacement Refrigerator,[501113]
93694,"[Sharkbite, Flexible, Connector, 18"", 3/4"",, S...",1573-000 Replacement Boiler/Water Heater,[402511]
89268,"[Universal, Mbox, Lock, Clip, On, Cylinder,, S...",5712-000 R&M - Hardware Locks & Keys,[925197]
100468,"[Edge, Banded, Door, and, Drawer, Fronts,, SKU...",5660-000 R&M - Windows & Doors,[597509]
79051,"[Sylvania, Flr, Bulb, 32w, T8, 5000k, Ll, 30/P...",5750-000 R&M - Electrical,[309113]
...,...,...,...
85672,"[30""white, Non, Vented, Range, Hood, W, Bracke...",5755-000 R&M - Appliances,[281200]
86456,"[Kwikset, Tylo, Passage, Knob, Satin, NICKEL,,...",5712-000 R&M - Hardware Locks & Keys,[817744]
110030,"[1000, Ml, PURELL, Nxt, Gel, Refill,, SKU=117890]",5605-000 Janitorial Service & Supplies,[117890]
75448,"[Two, Handle, Kitchen, Faucet, Cp, Dckplt, Chr...",5745-000 R&M - Plumbing,[158070]


Now, we can convert this column into a data type that we can use as a predictor.

Although, `SKU` is technically a categorical predictor and it's value is not measurable, its nominal order is important in the context of its proximity to other `SKU` values. Thus, we will change the datatype to `int`

In [7]:
for i in df.index:   
    df["SKU"].loc[i] = df["SKU"].loc[i][0]
df["SKU"] = df["SKU"].astype(int)
df = df.drop_duplicates(subset = ['SKU'])
df

,Description,Account,SKU
96874,"[Hotpoint, 17.5, Cu, Ft, Tp, Mnt, Rfrgrtr, Wht...",1557-000 Replacement Refrigerator,501113
93694,"[Sharkbite, Flexible, Connector, 18"", 3/4"",, S...",1573-000 Replacement Boiler/Water Heater,402511
89268,"[Universal, Mbox, Lock, Clip, On, Cylinder,, S...",5712-000 R&M - Hardware Locks & Keys,925197
100468,"[Edge, Banded, Door, and, Drawer, Fronts,, SKU...",5660-000 R&M - Windows & Doors,597509
79051,"[Sylvania, Flr, Bulb, 32w, T8, 5000k, Ll, 30/P...",5750-000 R&M - Electrical,309113
...,...,...,...
110266,"[E-z, Anchor, Stud, Solver, Pkg, Of, 100,, SKU...",5760-000 Maintenance Supplies,741413
109597,"[12, X, 6"", Three-way, Plastic, Register,, SKU...",5715-000 R&M - HVAC,253018
110030,"[1000, Ml, PURELL, Nxt, Gel, Refill,, SKU=117890]",5605-000 Janitorial Service & Supplies,117890
75448,"[Two, Handle, Kitchen, Faucet, Cp, Dckplt, Chr...",5745-000 R&M - Plumbing,158070


We can graph this data using Altair, with each `Account` number as it's x-axis and each product's (belonging to that account) `SKU` number as a y-value.

In [8]:
alt.data_transformers.disable_max_rows()
alt.Chart(df).mark_circle().encode(
    x = "Account",
    y = "SKU"
)

alt.Chart(...)

This is a lot of data to interpret, so we can simplify the data to the most common account numbers for the same of the project.

In [9]:
df["Account"].value_counts()

5745-000 R&M - Plumbing                     126
5750-000 R&M - Electrical                    77
5755-000 R&M - Appliances                    56
5760-000 Maintenance Supplies                55
5605-000 Janitorial Service & Supplies       54
5712-000 R&M - Hardware Locks & Keys         51
5660-000 R&M - Windows & Doors               32
5700-000 R&M - Interior                      29
5715-000 R&M - HVAC                          27
5623-000 Window Coverings                    24
5875-000 Paint & Decorating - Interior       17
5410-000 Office Supplies & Expense           15
5886-000 Tools & Equipment                   11
1558-000 Replacement Stove/Microwave          9
5725-000 R&M - Fire Life Safety               9
1557-000 Replacement Refrigerator             9
1559-000 Replacement Dishwasher               8
1560-000 Replacement Washer/Dryer             6
1573-000 Replacement Boiler/Water Heater      5
5635-000 R&M - Pool                           4
1556-000 Replacement A/C                

Let's calculate how much of the data lies in the 5 most populated `Account` values.

In [10]:
per = df["Account"].value_counts()[:5].sum()/df["Account"].value_counts().sum()
per

0.580441640378549

Approximately 60% of the data collected from the sample used belongs to one of the five most populated `Accounts`
We can now plot this graph.

In [11]:
df_top = df[df_pre["Account"].isin(df_pre["Account"].value_counts()[:5].index)].copy()
alt.data_transformers.disable_max_rows()
alt.Chart(df_top).mark_circle().encode(
    x = "Account",
    y = "SKU"
)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


alt.Chart(...)

Due to the magnitude of the data, it is still difficult to see any convincing trend relating `SKU` to `Account`
However, if we isolate smaller intervals of the data, we will see that proximity to certain `SKU` numbers can show a difference in distribution.

In [12]:
df_sub = df.loc[[i for i in df_top.index if df["SKU"].loc[i] in range(400000,402500)]]
df_sub.drop_duplicates(subset = ['SKU'])

,Description,Account,SKU
102975,"[1/2"", X, 1/4"", Angle, Stop,, SKU=400544]",5745-000 R&M - Plumbing,400544
90992,"[""1/2"""", X, 3/8"""", Angle, Stop"",, SKU=400542]",5745-000 R&M - Plumbing,400542
112807,"[Ptfe, Tape, 1/2"", X, 260"", Pkg, Of, 10,, SKU=...",5745-000 R&M - Plumbing,401600
76844,"[Sharkbite, Check, Valve, 3/4, "", Sb, X, Sb,, ...",5745-000 R&M - Plumbing,400548


In [13]:
preZoom = df_top["Account"].value_counts()["5745-000 R&M - Plumbing"]/len(df_top)
postZoom = df_sub["Account"].value_counts()[0]/len(df_sub)
print([preZoom,postZoom])
postZoom/preZoom

[0.3423913043478261, 1.0]


2.9206349206349205

Looking at the concentration of data, zooming into the interval of substrings `[400000,402500]`, we can compare the distribution of data to the entire sample of data. Precisely, whereas products under the `5745-000 R&M - Plumbing` account previously made up 31.8% of the data before, now 90.9% of data under a narrower interval of `SKU` numbers represents data belonging to `5745-000 R&M - Plumbing`

We will want to use a list of the `postZoom/preZoom` values for each account as our predictor in our model, so we need to create a few new columns:

In [14]:
df_top["Interval"] = [[df_top["SKU"].iloc[i]-1250,df_top["SKU"].iloc[i]+1250] for i in range(len(df_top))]
df_top

,Description,Account,SKU,Interval
79051,"[Sylvania, Flr, Bulb, 32w, T8, 5000k, Ll, 30/P...",5750-000 R&M - Electrical,309113,"[307863, 310363]"
91807,"[Replacement, 8, Surface, Element, Hds, 825479...",5755-000 R&M - Appliances,825479,"[824229, 826729]"
76274,"[Single, Handle, Kitchen, Faucet, Cp, Lss, Spr...",5745-000 R&M - Plumbing,158710,"[157460, 159960]"
110326,"[Pfstr, Vrvrplcmnt, Fct-shwr, Hdl, Chrm,, SKU=...",5745-000 R&M - Plumbing,442505,"[441255, 443755]"
110722,"[Sink, Bskt, 3-1/4"", Lug, Post, Ss, 5/Pkg,, SK...",5745-000 R&M - Plumbing,409850,"[408600, 411100]"
...,...,...,...,...
101222,"[PVC, Tubular, Tailpc, Flanged, 1, 1/2, X, 12,...",5745-000 R&M - Plumbing,403398,"[402148, 404648]"
110266,"[E-z, Anchor, Stud, Solver, Pkg, Of, 100,, SKU...",5760-000 Maintenance Supplies,741413,"[740163, 742663]"
110030,"[1000, Ml, PURELL, Nxt, Gel, Refill,, SKU=117890]",5605-000 Janitorial Service & Supplies,117890,"[116640, 119140]"
75448,"[Two, Handle, Kitchen, Faucet, Cp, Dckplt, Chr...",5745-000 R&M - Plumbing,158070,"[156820, 159320]"


Firstly, an interval columns allows us to make comparisons to other products whose SKU numbers are in close proximity to the one being analyzed.

Next, we can find the concentration of each Account among the products within the interval about the row at hand. This is then compared to the original proportion of the amount of products with a certain account compared to the entire dataset

In [15]:
!!! THIS CODE BLOCK MAY TAKE A WHILE TO RUN !!!

['/bin/bash: THIS: command not found']

In [16]:
compare = pd.Series(index = df_top["Account"].unique()).fillna(0)
cols = list(df_top["Account"].unique())
perc = np.array([df_top["Account"].value_counts()[k]/len(df_top) for k in range(len(df_top["Account"].unique()))])
new = list()
for row in range(len(df_top)):
    vals = np.array(list(df_top["Account"].loc[[sku for sku in df_top.index if df["SKU"].loc[sku] in range(df_top["Interval"].iloc[row][0],df_top["Interval"].iloc[row][1])]].value_counts().add(compare).fillna(0)))
    new.append(np.divide(vals/vals.sum(),perc))
df_top[df_top["Account"].unique()] = new
df_top

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


,Description,Account,SKU,Interval,5750-000 R&M - Electrical,5755-000 R&M - Appliances,5745-000 R&M - Plumbing,5605-000 Janitorial Service & Supplies,5760-000 Maintenance Supplies
79051,"[Sylvania, Flr, Bulb, 32w, T8, 5000k, Ll, 30/P...",5750-000 R&M - Electrical,309113,"[307863, 310363]",0.000000,0.000000,3.285714,0.000000,3.407407
91807,"[Replacement, 8, Surface, Element, Hds, 825479...",5755-000 R&M - Appliances,825479,"[824229, 826729]",0.000000,0.000000,0.000000,6.690909,0.000000
76274,"[Single, Handle, Kitchen, Faucet, Cp, Lss, Spr...",5745-000 R&M - Plumbing,158710,"[157460, 159960]",0.000000,2.730983,0.000000,0.000000,2.920635
110326,"[Pfstr, Vrvrplcmnt, Fct-shwr, Hdl, Chrm,, SKU=...",5745-000 R&M - Plumbing,442505,"[441255, 443755]",0.000000,4.779221,0.000000,0.000000,0.000000
110722,"[Sink, Bskt, 3-1/4"", Lug, Post, Ss, 5/Pkg,, SK...",5745-000 R&M - Plumbing,409850,"[408600, 411100]",0.000000,3.823377,0.000000,0.000000,1.362963
...,...,...,...,...,...,...,...,...,...
101222,"[PVC, Tubular, Tailpc, Flanged, 1, 1/2, X, 12,...",5745-000 R&M - Plumbing,403398,"[402148, 404648]",0.000000,4.779221,0.000000,0.000000,0.000000
110266,"[E-z, Anchor, Stud, Solver, Pkg, Of, 100,, SKU...",5760-000 Maintenance Supplies,741413,"[740163, 742663]",0.000000,0.000000,0.000000,0.000000,6.814815
110030,"[1000, Ml, PURELL, Nxt, Gel, Refill,, SKU=117890]",5605-000 Janitorial Service & Supplies,117890,"[116640, 119140]",2.628571,0.000000,0.000000,0.000000,0.681481
75448,"[Two, Handle, Kitchen, Faucet, Cp, Dckplt, Chr...",5745-000 R&M - Plumbing,158070,"[156820, 159320]",0.000000,3.186147,0.000000,0.000000,2.271605


Now, we have a parameter to use in a logistic regression model to classify our products based on the surrounding SKU numbers.

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [18]:
clf = LogisticRegression(max_iter=15000)
cols = list(df_top["Account"].unique())

In [19]:
X_train, X_test, y_train, y_test = train_test_split(df_top[cols], df_top["Account"])

In [20]:
clf.fit(X_train,y_train)
results = pd.DataFrame({'Predicition' : clf.predict(X_train),'Actual' : y_train, 'Correct': (clf.predict(X_train)==y_train)})
results

,Predicition,Actual,Correct
86033,5745-000 R&M - Plumbing,5745-000 R&M - Plumbing,True
75059,5745-000 R&M - Plumbing,5745-000 R&M - Plumbing,True
110326,5745-000 R&M - Plumbing,5745-000 R&M - Plumbing,True
91492,5750-000 R&M - Electrical,5750-000 R&M - Electrical,True
96462,5605-000 Janitorial Service & Supplies,5745-000 R&M - Plumbing,False
...,...,...,...
78492,5745-000 R&M - Plumbing,5745-000 R&M - Plumbing,True
84798,5745-000 R&M - Plumbing,5745-000 R&M - Plumbing,True
96585,5750-000 R&M - Electrical,5755-000 R&M - Appliances,False
96565,5745-000 R&M - Plumbing,5745-000 R&M - Plumbing,True


As you can see, the regression model is not perfect; however, it is able to make a solid prediction of the product's correct account, making this single predictor pretty strong.

With further exploration, machine learning could be used to interpret clues about the product's account based on the words in its description.

In [21]:
clf.score(X_train,y_train)

0.8297101449275363

In [22]:
clf.score(X_test,y_test)

0.7608695652173914

For now, though, the logistic regression model is able to predict with relative accuracy, correctly assigning an account more than 4 times as likely as a random guess would.

In pursuit of furthering my knowledge, I chose to research into the K-Nearest Neighbors algorithm, as it is very similar to my approach.

In [23]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(df_top[cols], df_top["Account"])

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train2,y_train2)
y_pred = knn.predict(X_test2)
accuracy = accuracy_score(y_test2, y_pred)
accuracy

0.7391304347826086

As it would seem, the model would find very similar accuracy compared to the Logistic Regression. Although, I do believe this comparison would vary with more quantitative data. 

## Summary

In this project, I manipulated a dataset with immeasurable data to classify products based on only their product ID. In the end, we were able to build a model with a success rate of around 80%. Although, for business purposes, I don't believe the model is ready to be used without supervision, it can be a great tool to determine a classification for a product whose description is hard to decipher. Further, I believe this specific model can be greatly enhanced with access to the HD Supply database, the store from which all these produts were purchased. 

## References

Your code above should include references.  Here is some additional space for references.

* What is the source of your dataset(s)?

I was given permission by my manager to use data related to our accounting database that records the product description, SKU number, and other inormation from our accounting workflow. This dataset itself is used as a reference for my work.

* List any other references that you found helpful.

https://scikit-learn.org/stable/modules/neighbors.html
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html
https://www.reddit.com/r/Python/comments/8jtwt8/pandas_multiple_column_value_assign/?rdt=54014
https://stackoverflow.com/questions/37693600/how-to-sort-dataframe-based-on-particular-stringcolumns-using-python-pandas

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ff38d052-3a93-4847-8449-1fe9db9b4dd0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>